### Projekt objektově orientovaného programování

---

Finální projekt kurzu Engeto, úvod do OOP.

<br>

Projekt **NENÍ** povinný. Každý, kdo má zájem, si jej může vyzkoušet.

<br>

Pokud budeš chtít zpětnou vazbou (konzultaci s lektorem), odevzdej, prosím, tvůj projekt jako repozitář přes *github* a na Slack pošli lektorovi url tohoto repozitáře.

<br>

Projekt bude mít tyto části:
1. [Sběr dat (OOP),](#Sběr-dat,-OOP, **povinné**)
    - [úvod k vlastnímu projektu,](#Úvod-k-vlastnímu-projektu)
    - [knihovny & výstup,](#Importování-knihoven-&-výstupní-objekt)
    - [vytvoření nového objektu,](#Vytvoření-objektu-pro-naše-data)
    - [parsování vrácených dat,](#Parsování-vrácených-dat)
    - [nové výstupní objekty,](#Vytvoření-pole-nových-objektů-typu-'BRRealEstateOffer')
    - [úkol A,](#Úkol-A)


2. [Analýza dat (pandas, volitelné),](#Analýza-data,-pandas)
    - [úvod,](#Úvod-k-frameworku)
    - [orientace,](#Zobrazení-údajů)
    - [selekce a řazení,](#Výběr-sloupce-&-seřazení)
    - [přidat & odstranit sloupec,](#Nové-sloupečky)
    - [úkol B,](#Úkol-B)


3. [Uložení dat (sqlite3, volitelné),](#Uložení-dat,-sqlite3,)
    - [úvod,](#Úvod-do-SQL)
    - [zápis testovacích hodnot,](#Tvorba-testovací-databáze-(orientace-v-DB))
    - [čtení testovacích hodnot,](#Čtení-hodnot-v-existující-DB)
    - [zápis skutečných hodnot,](#Zápis-hodnot-z-DataFramu-do-nové-tabulky)
    - [úkol C,](#Úkol-C)


4. [Interpretace dat (flask, ).](#Intepretace-pomocí-webového-frameworku)
    - [úvod,](#Úvod-do-webového-frameworku)
    - [views,hlavní stránka](#Hlavní-stránka,-první-část)
    - [aplikace dat,]()
    - [úkol D,](#Úkol-D)

<br>

#### Sběr dat, OOP

---

###### Úvod k vlastnímu projektu

---

Nachystání **virtuálního prostředí**:
```
$ python -m venv projekt04       # vytvořím virt. prostředí
$ source projekt04/bin/activate  # aktivuji binárku virt. prostředí
```

<br>

Instalace souvisejících **knihoven**:
```
$ pip --version  # kontrola manažeru
$ pip install -r requirements.txt

Collecting beautifulsoup4==4.10.0
  Using cached beautifulsoup4-4.10.0-py3-none-any.whl (97 kB)
Collecting certifi==2021.10.8
...
```

<br>

V rootovi projektu vytvoř soubor s dokumentací `README.md`.

<br>

Ten by měl obsahovat aspoň bodově **strukturu** tvého projektu, příklad objektů, se kterými bys chtěl pracovat a jednotlivé komponenty tvého projektu.

<br>

Vytvoř adresář pro nový balíček:
```
/<projekt04>
    ├─requirements.txt
    ├─README.md
    └─<jmeno_balicku>
       ├─__init__.py
       ├─collector.py
       ├─processor.py
       ├─db.py
       ├─tests/
       └─flask_fe/
          ├─static/
          ├─templates/
          ├─views.py
          └─db.db
```

###### Importování knihoven & výstupní objekt

---

In [24]:
"""
# náhled na údaj
# --------------
example: Dict[str, Union[str, float, int]] = 
{'gps': '{"lat":49.23727,"lng":16.58296}',
 'price': 6499000,
 'currency': 'CZK',
 'key_offer_type': 'prodej',
 'key_estate_type': 'byt',
 'key_disposition': '2-1',
 'surface': 60,
 'surface_land': 0}
"""

import json
from urllib.parse import urljoin
from geopy.geocoders import Nominatim
from typing import Dict, List, Union, Any

import bs4
import requests

<br>

###### Vytvoření objektu pro naše data

---

In [60]:
class BRRealEstateOffer:
    """Create a new object from the given attributes."""
    offer_count: int = 0
    
    def __init__(
        self, details: Dict[str, Union[str, int, float]]
    ):
        self.id_ = details.get("id")
        self.url = details.get("url")
        self.gps = details.get("gps")
        self.price = details.get("price")
        self.surface = details.get("surface")
        self.currency = details.get("currency")
        self.surface_land = details.get("surfaceLand")
        self.key_offer_type = details.get("keyOfferType")
        self.key_estate_type = details.get("keyEstateType")
        self.key_disposition = details.get("keyDisposition")

    @classmethod
    def add_offer(cls):
        cls.offer_count += 1

    def __repr__(self) -> str:
        return str(f"{self.url}")
    
    # ...
    def full_description(self) -> str:
        return f"{self.key_offer_type}; {self.key_estate_type}; {self.key_disposition}"

###### Vzorový údaj po zapracování

---

In [61]:
example: Dict[str, Union[str, float, int]] = {
    'id': 695305,
    'surface': 60,
    'price': 6499000,
    'currency': 'CZK',
    'surface_land': 0,
    'keyEstateType': 'byt',
    'keyDisposition': '2-1',
    'keyOfferType': 'prodej',
    'gps': '{"lat":49.23727,"lng":16.58296}',
    'url': "https://www.bezrealitky.cz/nemovitosti-byty-domy/695305-nabidka-prodej-bytu-ostruzinova-brno",
}

In [62]:
offer_1 = BRRealEstateOffer(example)

In [63]:
offer_1.full_description()

'prodej; byt; 2-1'

###### Požadavek na API

---

In [64]:
class ScraperInitiator:
    """Initiate a new object for the data transfer."""
    
    def __init__(self, url: str, params: Dict[str, str]):
        self.url = url
        self.params = params
        
    def send_post_request(self) -> requests.models.Response:
        return requests.post(self.url, params=self.params)
    
    @staticmethod
    def load_json(response: requests.models.Response) -> List[Dict[str, str]]:
        """Load the 'json' package and read the content from string."""
        return json.loads(response.text)

In [65]:
session_1 = ScraperInitiator(
    "https://www.bezrealitky.cz/api/record/markers",
    {
        'offerType': 'prodej',
        'submit': '1',
        'boundary': '[[[{"lat":52,"lng":12},{"lat":52,"lng":16},{"lat":50,"lng":16},{"lat":50,"lng":12},{"lat":52,"lng":12}]]]'
    }
)

In [66]:
json_: List[Dict[str, Any]] = session_1.load_json(
    session_1.send_post_request()
)

In [67]:
type(json_)

list

In [68]:
json_[0]

{'id': '695659',
 'uri': '695659-nabidka-prodej-garaze-stechovicka-hlavni-mesto-praha',
 'keyAdvertType': 'estate_offer',
 'type': '',
 'timeOrder': {'date': '2021-12-09 11:02:45.000000',
  'timezone_type': 3,
  'timezone': 'Europe/Berlin'},
 'orderPriority': 0,
 'advertEstateOffer': [{'gps': '{"lat":50.07153,"lng":14.50459}',
   'price': 790000,
   'currency': 'CZK',
   'keyOfferType': 'prodej',
   'keyEstateType': 'garaz',
   'keyDisposition': '',
   'surface': 15,
   'surfaceLand': 0,
   'id': '695659'}]}

<br>

###### Parsování vrácených dat

---

In [69]:
class DataParser:
    """Parse the given data and create cleaner, non-nested python object."""
    
    def __init__(self, data: List[Dict[str, Any]]):
        self.data = data
        self.url: str = "https://www.bezrealitky.cz/nemovitosti-byty-domy/"
        
    def iterate_through_data(self):
        results: list = []
        
        for offer in self.data:
            uri, details = self.parse_main_dict(offer)
            results.append(
                self.extend_dict(url=urljoin(self.url, uri), details=details)
            )
        
        return results

    @staticmethod
    def parse_main_dict(offer: Dict[str, Any]) -> tuple:
        """Parse and return attributes uri, advertEstateOffer."""
        return offer.get("uri"), offer.get("advertEstateOffer")[0]

    @staticmethod
    def parse_nested_dict(details: Dict[str, Union[str, int]]):
        """Parse and return attributes from the given dictionary."""
        return {
            "gps": details.get("gps"),
            "price": details.get("price"),
            "currency": details.get("currency"),
            "key_offer_type": details.get("keyOfferType"),
            "key_estate_type": details.get("keyEstateType"),
            "key_disposition": details.get("keyDisposition"),
            "surface": details.get("surface"),
            "surface_land": details.get("surfaceLand"),
            "surface": details.get("surface"),
        }
    
    @staticmethod
    def extend_dict(**kwargs):
        """Create and update a new dictionary object with the given attrs."""
        attributes: Dict[str, str] = {}

        for key, val in kwargs.items():
            if key == "url":
                attributes[key] = val 
            else:
                attributes.update(val)
                
        return attributes

In [70]:
parser_1 = DataParser(json_)

In [71]:
testing_list = parser_1.iterate_through_data()

In [72]:
testing_list[1]

{'url': 'https://www.bezrealitky.cz/nemovitosti-byty-domy/695657-nabidka-prodej-bytu-v-lesicku-hlavni-mesto-praha',
 'gps': '{"lat":50.07825,"lng":14.40628}',
 'price': 12950000,
 'currency': 'CZK',
 'keyOfferType': 'prodej',
 'keyEstateType': 'byt',
 'keyDisposition': '3-kk',
 'surface': 73,
 'surfaceLand': 0,
 'id': '695657'}

###### Vytvoření pole nových objektů typu 'BRRealEstateOffer'

---

In [73]:
class BRRealEstateOfferProcessor:
    """Process the given attributes and a new object 'BRRealEstateOffer'."""
    
    def __init__(self, parsed_data: List[Dict[str, Union[str, int]]]):
        self.offers: list = []
        self.parsed_data = parsed_data

    def add_offer(self, dict_data: Dict[str, Union[str, int]]) -> None:
        self.offers.append(
            BRRealEstateOffer(dict_data)
        )

    def process_offers(self) -> None:
        for offer in self.parsed_data:
            self.add_offer(offer)

In [74]:
proc_1 = BRRealEstateOfferProcessor(testing_list)
proc_1.process_offers()

In [79]:
proc_1.offers[500].full_description()

'prodej; dum; ostatni'

###### Úkol A

---

Zkus si třídy a instancování z předchozí sekce zapsat do jednoho spustitelného modulu.

<br>

Příklad spuštění v příkazovém řádku:
```
>>> from bezrealitky.collector import run_collector
>>> offers = run_collector()
>>> offers[0]
https://www.bezrealitky.cz/nemovitosti-byty-domy/695659-nabidka-prodej-garaze-stechovicka-hlavni-mesto-praha
```

<br>

#### Analýza data, pandas

---

<br>

###### Úvod k frameworku

---
Obecný popisek [zde](https://pypi.org/project/pandas/)

<br>

Nahrátí knihovny:

In [83]:
import pandas
# import pandas as pd
# from pandas import DataFrame

<br>

Vytvoření DataFrame:

In [87]:
br_dataframe: pandas.core.frame.DataFrame = pandas.DataFrame(testing_list)

<br>

###### Zobrazení údajů

---

In [88]:
br_dataframe.head()  # 5 prvni udaju

,url,gps,price,currency,keyOfferType,keyEstateType,keyDisposition,surface,surfaceLand,id
0,https://www.bezrealitky.cz/nemovitosti-byty-do...,"{""lat"":50.07153,""lng"":14.50459}",790000,CZK,prodej,garaz,,15,0,695659
1,https://www.bezrealitky.cz/nemovitosti-byty-do...,"{""lat"":50.07825,""lng"":14.40628}",12950000,CZK,prodej,byt,3-kk,73,0,695657
2,https://www.bezrealitky.cz/nemovitosti-byty-do...,"{""lat"":50.143740650145524,""lng"":15.15070511182...",4320000,CZK,prodej,pozemek,,703,0,695646
3,https://www.bezrealitky.cz/nemovitosti-byty-do...,"{""lat"":50.025349500000004,""lng"":14.43243270000...",6800000,CZK,prodej,byt,3-1,76,0,695629
4,https://www.bezrealitky.cz/nemovitosti-byty-do...,"{""lat"":50.02651,""lng"":14.22488}",5600000,CZK,prodej,byt,3-kk,61,0,695618


In [94]:
br_dataframe.describe()

,price,surface,surfaceLand
count,9.730000e+02,973.000000,973.000000
mean,7.498723e+06,941.105858,252.112025
std,6.592643e+06,8223.908034,909.838653
min,0.000000e+00,0.000000,0.000000
25%,3.549000e+06,50.000000,0.000000
50%,6.190000e+06,73.000000,0.000000
75%,9.200000e+06,143.000000,0.000000
max,8.366280e+07,196860.000000,11606.000000


<br>

###### Výběr sloupce & seřazení

---

In [109]:
br_dataframe.loc[:10, ["url", "price"]]  # selekce řádků a sloupců

,url,price
0,https://www.bezrealitky.cz/nemovitosti-byty-do...,790000
1,https://www.bezrealitky.cz/nemovitosti-byty-do...,12950000
2,https://www.bezrealitky.cz/nemovitosti-byty-do...,4320000
3,https://www.bezrealitky.cz/nemovitosti-byty-do...,6800000
4,https://www.bezrealitky.cz/nemovitosti-byty-do...,5600000
5,https://www.bezrealitky.cz/nemovitosti-byty-do...,6100000
6,https://www.bezrealitky.cz/nemovitosti-byty-do...,3850000
7,https://www.bezrealitky.cz/nemovitosti-byty-do...,6700000
8,https://www.bezrealitky.cz/nemovitosti-byty-do...,2980000
9,https://www.bezrealitky.cz/nemovitosti-byty-do...,2500000


In [111]:
br_dataframe.iloc[:10, 0:3]  # selekce řádků a sloupců

,url,gps,price
0,https://www.bezrealitky.cz/nemovitosti-byty-do...,"{""lat"":50.07153,""lng"":14.50459}",790000
1,https://www.bezrealitky.cz/nemovitosti-byty-do...,"{""lat"":50.07825,""lng"":14.40628}",12950000
2,https://www.bezrealitky.cz/nemovitosti-byty-do...,"{""lat"":50.143740650145524,""lng"":15.15070511182...",4320000
3,https://www.bezrealitky.cz/nemovitosti-byty-do...,"{""lat"":50.025349500000004,""lng"":14.43243270000...",6800000
4,https://www.bezrealitky.cz/nemovitosti-byty-do...,"{""lat"":50.02651,""lng"":14.22488}",5600000
5,https://www.bezrealitky.cz/nemovitosti-byty-do...,"{""lat"":50.02914,""lng"":14.53507}",6100000
6,https://www.bezrealitky.cz/nemovitosti-byty-do...,"{""lat"":50.6454,""lng"":14.04379}",3850000
7,https://www.bezrealitky.cz/nemovitosti-byty-do...,"{""lat"":50.72607,""lng"":15.17718}",6700000
8,https://www.bezrealitky.cz/nemovitosti-byty-do...,"{""lat"":50.25203579999999,""lng"":14.5187024}",2980000
9,https://www.bezrealitky.cz/nemovitosti-byty-do...,"{""lat"":50.07845,""lng"":12.37209}",2500000


In [112]:
br_dataframe.sort_values(by="price", ascending=True)  # seřaď podle nejvyšší ceny

,url,gps,price,currency,keyOfferType,keyEstateType,keyDisposition,surface,surfaceLand,id
497,https://www.bezrealitky.cz/nemovitosti-byty-do...,"{""lat"":50.1264298,""lng"":14.5766071}",0,CZK,prodej,dum,4-kk,140,222,690044
143,https://www.bezrealitky.cz/nemovitosti-byty-do...,"{""lat"":50.0563843,""lng"":14.5144117}",0,CZK,prodej,byt,2-kk,77,0,694301
20,https://www.bezrealitky.cz/nemovitosti-byty-do...,"{""lat"":50.41872,""lng"":12.9821}",1,CZK,prodej,pozemek,,700,0,695445
131,https://www.bezrealitky.cz/nemovitosti-byty-do...,"{""lat"":50.07104,""lng"":14.48376}",1,CZK,prodej,dum,ostatni,220,220,694411
725,https://www.bezrealitky.cz/nemovitosti-byty-do...,"{""lat"":50.2287795,""lng"":14.3575719}",500,CZK,prodej,pozemek,,9044,0,683328
...,...,...,...,...,...,...,...,...,...,...
480,https://www.bezrealitky.cz/nemovitosti-byty-do...,"{""lat"":50.1026307,""lng"":14.3846884}",32000000,CZK,prodej,dum,ostatni,264,211,690282
249,https://www.bezrealitky.cz/nemovitosti-byty-do...,"{""lat"":50.07893,""lng"":14.42453}",42900000,CZK,prodej,byt,ostatni,211,0,693124
780,https://www.bezrealitky.cz/nemovitosti-byty-do...,"{""lat"":50.7554272,""lng"":15.0644769}",49440000,CZK,prodej,kancelar,,1493,1720,678022
439,https://www.bezrealitky.cz/nemovitosti-byty-do...,"{""lat"":50.08084003981505,""lng"":14.485899098217...",55000000,CZK,prodej,dum,ostatni,1000,6000,691157


In [127]:
br_dataframe[br_dataframe["price"] == 0]

,url,gps,price,currency,keyOfferType,keyEstateType,keyDisposition,surface,surfaceLand,id
143,https://www.bezrealitky.cz/nemovitosti-byty-domy/694301-nabidka-prodej-bytu-svehlova,"{""lat"":50.0563843,""lng"":14.5144117}",0,CZK,prodej,byt,2-kk,77,0,694301
497,https://www.bezrealitky.cz/nemovitosti-byty-domy/690044-nabidka-prodej-domu,"{""lat"":50.1264298,""lng"":14.5766071}",0,CZK,prodej,dum,4-kk,140,222,690044


In [116]:
pandas.set_option('display.max_colwidth', 150)

###### Nové sloupečky

---

In [134]:
br_dataframe["type_upper"] = [offer.upper() for offer in br_dataframe["keyEstateType"]]

In [ ]:
br_dataframe.drop("type_upper", axis=1, inplace=True)

In [59]:
br_dataframe["formatted_price"] = [f"{x:.2f} mil." for x in br_dataframe['price'] / 1000000]

<br>

###### Naformátuj mi cenu na čitelnější hodnotu a ulož do nového sloupce

---

In [66]:
br_dataframe.drop("formatted_price[mil]", axis=1, inplace=True)


###### Vypiš všechny typy nabídek nemovitostí (unikátní hodnoty) a jejich výskyt

---

In [71]:
br_dataframe["keyEstateType"].unique()

array(['byt', 'dum', 'pozemek', 'nebytovy-prostor', 'garaz',
       'rekreacni-objekt', 'kancelar'], dtype=object)

In [144]:
for estate_type in br_dataframe["keyEstateType"].unique():
    print(f"TYPE: {estate_type:^16}; \
COUNT: {br_dataframe['keyEstateType'].eq(estate_type).sum()}"
    )

TYPE:      garaz      ; COUNT: 21
TYPE:       byt       ; COUNT: 600
TYPE:     pozemek     ; COUNT: 102
TYPE:       dum       ; COUNT: 183
TYPE: nebytovy-prostor; COUNT: 22
TYPE: rekreacni-objekt; COUNT: 35
TYPE:     kancelar    ; COUNT: 10


In [137]:
estate_type = pandas.Series(br_dataframe["keyEstateType"])

In [141]:
estate_type

0        garaz
1          byt
2      pozemek
3          byt
4          byt
        ...   
968        byt
969        dum
970    pozemek
971        byt
972        byt
Name: keyEstateType, Length: 973, dtype: object

In [138]:
estate_type.unique()

array(['garaz', 'byt', 'pozemek', 'dum', 'nebytovy-prostor',
       'rekreacni-objekt', 'kancelar'], dtype=object)


###### Vypiš mi nejdražší a nejlevnější nemovitost

---

<br>

Přilož detail z odkazem, ať můžeš nabídku zkontrolovat:

In [149]:
br_dataframe["price"].max()
# br_dataframe[br_dataframe["price"] == 0]
# br_dataframe[br_dataframe["price"] > 50000000]

,url,gps,price,currency,keyOfferType,keyEstateType,keyDisposition,surface,surfaceLand,id,JeByt,type_upper
439,https://www.bezrealitky.cz/nemovitosti-byty-domy/691157-nabidka-prodej-domu-praha-10,"{""lat"":50.08084003981505,""lng"":14.485899098217489}",55000000,CZK,prodej,dum,ostatni,1000,6000,691157,Dum,DUM
670,https://www.bezrealitky.cz/nemovitosti-byty-domy/685677-nabidka-prodej-pozemku-ruska-frantiskovy-lazne,"{""lat"":50.1214475,""lng"":12.3610505}",83662800,CZK,prodej,pozemek,,53976,0,685677,Pozemek,POZEMEK



###### Vytvoř sloupec, který bude obsahovat skutečnou adresu (podle GPS)

---

<br>

Doplň sloupec do DataFramu, který bude obsahovat celou dostupnou adresu:

In [150]:
def replace_gps(gps: str) -> str:
    """Find the address with the gps coordinates."""
    
    lat, lng = json.loads(gps).values()
    join_coords = ", ".join((str(lat), str(lng)))

    geolocator = Nominatim(
        user_agent="gps_convertor"
    )
        
    return geolocator.reverse(join_coords).address

# address = replace_gps('{"lat":50.25203579999999,"lng":14.5187024}')

In [151]:
# address

'1044, Kojetická, Neratovice, okres Mělník, Střední Čechy, 27711, Česko'

In [158]:
br_dataframe.head()

,url,gps,price,currency,keyOfferType,keyEstateType,keyDisposition,surface,surfaceLand,id,formatted_price,full_address
0,https://www.bezrealitky.cz/nemovitosti-byty-domy/695563-nabidka-prodej-bytu-kojeticka,"{""lat"":50.25203579999999,""lng"":14.5187024}",2980000,CZK,prodej,byt,2-kk,40,0,695563,2.98 mil.,"1044, Kojetická, Neratovice, okres Mělník, Střední Čechy, 27711, Česko"
1,https://www.bezrealitky.cz/nemovitosti-byty-domy/695544-nabidka-prodej-bytu-skolni-cheb,"{""lat"":50.07845,""lng"":12.37209}",2500000,CZK,prodej,byt,2-1,55,0,695544,2.50 mil.,"653/17, Školní, Cheb, okres Cheb, Karlovarský kraj, Severozápad, 35002, Česko"
2,https://www.bezrealitky.cz/nemovitosti-byty-domy/695541-nabidka-prodej-domu,"{""lat"":50.0485532,""lng"":14.5241392}",19600000,CZK,prodej,dum,5-kk,180,180,695541,19.60 mil.,"50/15, Chalupnická, Na Košíku, Hostivař, Hlavní město Praha, Praha, 10200, Česko"
3,https://www.bezrealitky.cz/nemovitosti-byty-domy/695540-nabidka-prodej-domu,"{""lat"":50.0853554,""lng"":14.1300829}",13520000,CZK,prodej,dum,,159,400,695540,13.52 mil.,"108, nám. T. G. Masaryka, Unhošť, okres Kladno, Střední Čechy, 27351, Česko"
4,https://www.bezrealitky.cz/nemovitosti-byty-domy/695535-nabidka-prodej-pozemku-usti-nad-labem,"{""lat"":50.7020975,""lng"":14.0031103}",4098980,CZK,prodej,pozemek,,1244,0,695535,4.10 mil.,"Habrovice, Ústí nad Labem, okres Ústí nad Labem, Ústecký kraj, Severozápad, 40340, Česko"


###### Úkol B

---
1. Zjistit typy nabízených nemovitostí & jejich výskyt, nejvyšší & nejnižší ceny,

<br>

2. Zkus přidat nový sloupeček `formatted_price`, který bude obsahovat naformátovanou cenu<br>
    ```int64: 2980000 -> object: 2.98```

<br>

3. Zkus přidat nový sloupeček `full_address`, kde předáš DataFramu skutečnou adresu, na základě GPS.

<br>

#### Uložení dat, sqlite3,

---

<br>

###### Úvod do SQL

---

Oficiální dokumenta pro SQLite [zde](https://sqlite.org/docs.html).

<br>

**Ano**:
- jednoduché, inituitivní,
- zdarma (většinou),
- serverless,
- `null`, `integer`, `text`, `real`,

**Ne**:
- neefektivní práce s většími daty (1TB+ ukládat na centralizované db),
- umožňuje, aby v daném okamžiku proběhla pouze jedna operace zápisu ,

<br>



###### Zápis do testovací databáze

---

In [201]:
import sqlite3

In [202]:
# zajistím spojení s novým objektem db
con = sqlite3.connect('testing_db.db')

In [203]:
# vytvořím objekt kurzoru, skrze který provádím jednotlivé dotazy
cur = con.cursor()

In [204]:
# vytvořím novou testovací tabulku, v případě že žádná neexistuje
cur.execute('''
CREATE TABLE IF NOT EXISTS example (
id INTEGER PRIMARY KEY,
name TEXT,
email TEXT)
''')

In [205]:
# Vložím tři různé řádky
cur.execute("INSERT INTO example VALUES (NULL,'Matous','matous@matous.cz')")
cur.execute("INSERT INTO example VALUES (NULL,'Petr','petr@metr.cz')")
cur.execute("INSERT INTO example VALUES (NULL,'Lukas','luki@puky.cz')")

# commitnu změny do tabulky
con.commit()

# ukončím spojení
con.close()

In [206]:
!ls -l

total 220
-rw-rw-r-- 1 matous matous 20475 lis 30 18:43 lesson00.ipynb
-rw-rw-r-- 1 matous matous 24891 pro  5 10:58 lesson01.ipynb
-rw-rw-r-- 1 matous matous 41117 pro  6 12:05 lesson02.ipynb
-rw-rw-r-- 1 matous matous 45374 pro  8 11:05 lesson03.ipynb
-rw-rw-r-- 1 matous matous 73251 pro  9 14:06 lesson04.ipynb
-rw-r--r-- 1 matous matous  8192 pro  9 14:07 testing_db.db


###### Zápis pomocí sqlite

---

In [208]:
# zajistím spojení s novým objektem db
con = sqlite3.connect('testing_db.db')
cur = con.cursor()

sl_ = {
    "name": "Marek",
    "email": "marek.parek@email.cz"
}

cur.execute(f"""INSERT INTO example ({','.join(sl_.keys())}) \
            VALUES ({','.join(['?'] * len(sl_))})""", tuple(sl_.values()))

con.commit()

# ukončím spojení
con.close()

###### Čtení hodnot v existující DB

---

In [167]:
con = sqlite3.connect('testing_db.db')
cur = con.cursor()

# označím všechny sloupce v  tabulce 'testing_db.sql'
cur.execute("SELECT * FROM example")

# nainicializuji je do proměnné 'data'
data = cur.fetchall()

# vypisuji řádek po řádku pomocí iterování skrze objekt
for row in data:
    print(row)

con.close()

(1, 'Matous', 'matous@matous.cz')
(2, 'Petr', 'petr@metr.cz')
(3, 'Lukas', 'luki@puky.cz')


###### Context manager

---

In [168]:
with sqlite3.connect('testing_db.sql') as con:
    cur = con.cursor()
    
    cur.execute("SELECT * FROM example")
    data = cur.fetchall()
    for row in data:
        print(row)

(1, 'Matous', 'matous@matous.cz')
(2, 'Petr', 'petr@metr.cz')
(3, 'Lukas', 'luki@puky.cz')


In [170]:
br_dataframe.rename(columns = {'id':'offer_id'}, inplace = True)

In [179]:
br_dataframe["offer_id"].nunique()

981

###### Zápis hodnot z DataFramu do nové tabulky

---

In [210]:
with sqlite3.connect('br_offers.db') as con:
    cur = con.cursor()
    cur.execute("""
    CREATE TABLE IF NOT EXISTS offers (
    id INTEGER PRIMARY KEY,
    url TEXT,
    gps TEXT,
    price INTEGER,
    currency TEXT,
    keyOfferType TEXT,
    keyEstateType TEXT,
    keyDisposition TEXT,
    surface INTEGER,
    surfaceLand INTEGER,
    offer_id INTEGER,
    formatted_price TEXT,
    full_address TEXT
    )
    """
    )

    con.commit()

    br_dataframe.to_sql(
        'offers', con, if_exists='replace',
        index=True,
#         index_label="id"
    )

###### Úkol C

---

Zkus zapsat získaná data pomocí objektu v Pythonu jako databázový systém.

<br>

#### Intepretace pomocí webového frameworku

---

<br>

###### Úvod do webového frameworku

---

Dokumentaci lze najít [zde](https://flask.palletsprojects.com/en/2.0.x/)

<br>



###### Hlavní stránka

---

Výstup:
```
Ahoj, na moji hlavni strance!
```

In [214]:
# flask_app.py bez .html
from flask import Flask  # naimportuj třídu 'Flask'

app = Flask(__name__)    # vytvoříme instanci se jménem akt. souboru 

@app.route("/")          # pomocí dekorátoru vytvořím cílové url
def main_page():         # funkce pro úvodní stránku
    return "<p>Ahoj, na moji hlavni strance</p>"

# if __name__ == "__main__":
#     app.run()

In [215]:
# flask_app.py s templates/main_page.html
from flask import Flask, render_template

app = Flask(__name__)


@app.route("/")
def main_page():
    return render_template("main_page.html")


# if __name__ == "__main__":
#     app.run(debug=True)    # po každé úpravě dojde na refresh

###### HTML main_page

---
```html
<!-- main_page.html -->
<html>
  <head>
    <title>Main page</title>
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css" integrity="sha384-ggOyR0iXCbMQv3Xipma34MD+dH/1fQ784/j6cY/iJTQUOhcWr7x9JvoRxT2MZw1T" crossorigin="anonymous">
  </head>
<body>
    <div class="row-3" id="br_offers">
      <div class="row 4">
        <h1 id="title">Ahoj na mojí hlavní stránce!</h1>
      </div>
    </div>
</body
```

###### Stránka s výsledky

---

In [216]:
# main_page.py
import os
import sqlite3

from flask import Flask, render_template, request

app = Flask(__name__)


@app.route("/")
def main_page():
    return render_template("main_page.html")


@app.route("/results")
def results():
    with sqlite3.connect("01_testing.db") as con:
        cur = con.cursor()

        cur.execute("SELECT * FROM example")
        data = cur.fetchall()

    return render_template("results.html", result=data)

###### HTML results.html
---

Výstup:
```
(1, 'Matous', 'matous@matous.cz')
(2, 'Petr', 'petr@metr.cz')
(3, 'Lukas', 'luki@puky.cz')
(4, 'Marek', 'marek.parek@email.cz')
```


<br>

```html
<!-- results.html -->
<html>
  <head>
    <title>Results</title>
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css" integrity="sha384-ggOyR0iXCbMQv3Xipma34MD+dH/1fQ784/j6cY/iJTQUOhcWr7x9JvoRxT2MZw1T" crossorigin="anonymous">
  </head>
<body>
    <div class="container" id="results">

      <div class="row 4">
        <h1 id="title">Výsledky</h1>
      </div>

      <div class="row 3">
        <table id="ner-table" data-test-id="ner-table" class="table table-bordered">
          <thead class="thead-dark">
            <tr>
              <th>id</th>
              <th>jmeno</th>
              <th>email</th>
            </tr>
          </thead>
          <tbody>
            {% for row in result %}
            <tr>
                <td>{{ row }}</td>
            </tr>
            {% endfor %}
          </tbody>
        </table>
      </div>
    </div>
  </body>
</html>
```


###### Doplníme indexy

---

Výstup:
```
1 	Matous 	matous@matous.cz
2 	Petr 	petr@metr.cz
3 	Lukas 	luki@puky.cz
4 	Marek 	marek.parek@email.cz
```

<br>

```html
          <thead class="thead-dark">
            <tr>
              <th>id</th>
              <th>jmeno</th>
              <th>email</th>
            </tr>
          </thead>
          <tbody>
            {% for row in result %}
            <tr>
                <td>{{ row[0] }}</td>
                <td>{{ row[1] }}</td>
                <td>{{ row[2] }}</td>
            </tr>
            {% endfor %}
          </tbody>
```

```html
<html>
  <head>
    <title>Results</title>
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css" integrity="sha384-ggOyR0iXCbMQv3Xipma34MD+dH/1fQ784/j6cY/iJTQUOhcWr7x9JvoRxT2MZw1T" crossorigin="anonymous">
  </head>
<body>
    <div class="container p-3" id="app">

      <div class="row my-4">
        <h1 id="app-title">Results</h1>
      </div>

      <div class="row my-3">
        <table id="ner-table" data-test-id="ner-table" class="table table-bordered">
          <thead class="thead-dark">
            <tr>
              <th>id</th>
              <th>url</th>
              <th>price</th>
              <th>currency</th>
              <th>type</th>
              <th>surface</th>
            </tr>
          </thead>
          <tbody>
            {% for row in result %}
            <tr>
                <td>{{ row[10] }}</td>
                <td>{{ row[1] }}</td>
                <td>{{ row[3] }}</td>
                <td>{{ row[4] }}</td>
                <td>{{ row[6] }}</td>
                <td>{{ row[8] }}</td>
            </tr>
            {% endfor %}
          </tbody>
        </table>
      </div>
    </div>
  </body>
</html>
```

###### Úkol D

---

Zkus zobrazit získaná data pomocí frameworku *Flask* na localhostu. Rozděl jednotlivé údaje do tabulky a vypiš některé hodnoty.

---